# Fine Tuning Small Language Model on AG News Dataset

This notebook demonstrates fine-tuning of a Small Language Model (DistilBERT) using AG News dataset from Hugging Face. The model is trained to classify news articles into four categories.


In [16]:
!pip install transformers datasets evaluate accelerate


In [27]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

## Dataset Selection

AG News dataset is used for multi-class news classification. It contains news articles categorized into World, Sports, Business, and Science/Technology.



In [18]:
dataset = load_dataset("ag_news")

dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [19]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [20]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

## Model Selection

DistilBERT is chosen because it is a lightweight Small Language Model that provides high performance with faster training time.



In [21]:
model = AutoModelForSequenceClassification.from_pretrained(
      "distilbert-base-uncased",
          num_labels=4
          )



Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [22]:
metric = evaluate.load("accuracy")


In [23]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [24]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [28]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(3000)),
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(1000)),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

The pre-trained DistilBERT model is fine-tuned using AG News dataset to adapt it for news classification.

In [29]:
trainer.train()

Step,Training Loss
500,0.454797


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=750, training_loss=0.3967182820638021, metrics={'train_runtime': 78.4604, 'train_samples_per_second': 76.472, 'train_steps_per_second': 9.559, 'total_flos': 127249823902656.0, 'train_loss': 0.3967182820638021, 'epoch': 2.0})

## Model Evaluation

Accuracy metric is used to evaluate classification performance.


In [30]:
trainer.evaluate()


{'eval_loss': 0.37376636266708374,
 'eval_accuracy': 0.896,
 'eval_runtime': 2.7023,
 'eval_samples_per_second': 370.059,
 'eval_steps_per_second': 46.257,
 'epoch': 2.0}

## Results

The model achieved approximately 89% accuracy on the evaluation dataset. The training loss reduced steadily indicating successful learning.

## Observations

- Fine-tuning improves classification accuracy.
- Pre-trained models reduce training time.
- Increasing training epochs can further improve results.


In [33]:
text = "NASA launches a new satellite for space research"

inputs = tokenizer(text, return_tensors="pt")
# Move inputs to the same device as the model
inputs = {k: v.to(model.device) for k, v in inputs.items()}
outputs = model(**inputs)

prediction = np.argmax(outputs.logits.detach().cpu().numpy())

labels = ["World", "Sports", "Business", "Sci/Tech"]
print("Predicted Category:", labels[prediction])

Predicted Category: Sci/Tech


In [34]:
labels = ["World", "Sports", "Business", "Sci/Tech"]

print("Predicted Category:", labels[prediction])


Predicted Category: Sci/Tech


In [36]:
from google.colab import files
files.upload()


Saving Fine_tuning_Lab_task1 (1).ipynb to Fine_tuning_Lab_task1 (1).ipynb


{'Fine_tuning_Lab_task1 (1).ipynb': b'{\n  "nbformat": 4,\n  "nbformat_minor": 0,\n  "metadata": {\n    "colab": {\n      "provenance": [],\n      "gpuType": "T4"\n    },\n    "kernelspec": {\n      "name": "python3",\n      "display_name": "Python 3"\n    },\n    "language_info": {\n      "name": "python"\n    },\n    "accelerator": "GPU",\n    "widgets": {\n      "application/vnd.jupyter.widget-state+json": {\n        "a46c9d063e114ec3b575453a8cc4c430": {\n          "model_module": "@jupyter-widgets/controls",\n          "model_name": "HBoxModel",\n          "model_module_version": "1.5.0",\n          "state": {\n            "_dom_classes": [],\n            "_model_module": "@jupyter-widgets/controls",\n            "_model_module_version": "1.5.0",\n            "_model_name": "HBoxModel",\n            "_view_count": null,\n            "_view_module": "@jupyter-widgets/controls",\n            "_view_module_version": "1.5.0",\n            "_view_name": "HBoxView",\n            "box_sty

In [45]:
import nbformat

path = "/content/Fine_tuning_Lab_task1 (1).ipynb"

# Read notebook
with open(path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

    # Remove widgets metadata
    if "widgets" in nb.get("metadata", {}):
        del nb["metadata"]["widgets"]

        # Write cleaned notebook
        with open("/content/Fine_tuning_Lab_task1_clean.ipynb", "w", encoding="utf-8") as f:
            nbformat.write(nb, f)

            print("Notebook cleaned successfully!")
